In [ ]:
# read in P100 Excel spreadsheet and determine jobs that extend several days
# Method: for ease of i/o in and out of the dataframe, make an adjacent dataframe to 
# hold 'new' records for jobs that are multiday and append as we go, multirecords created
# as needed for date accounting - basically so that using start_date as the record to track the day or day(s) 
# that a job ran.

In [ ]:
import pandas as pd
import numpy as np
import math
import bokeh
from bokeh.core.properties import Enum 
from bokeh.plotting import figure, ColumnDataSource, show 
from bokeh.models import ColumnDataSource, HoverTool
import datetime
from datetime import timedelta

In [ ]:
# Read in an abbreviated version of Mike's spreadsheet.  NOTE a few headers have been edited/added from the original
# spreadsheet.  This is the only 'hardcoded' line pointing to a file.

# optional var to set num rows to read 
#numrows = 2000

P100data = pd.read_excel('/Users/cyndy/Documents/HPCstats/USEME_P100.xlsx', 
                   sheet_name=('P100'), 
                   usecols=['user','group', 'session', 'start', 'end', 'startdate', 'enddate', 'start_date', 
                  'end_date', 'days', 'hours', 'ncpus'], 
                   converters = {'start':pd.to_numeric, 'end':pd.to_numeric, 'startdate':pd.to_datetime})

In [ ]:
numrecords = P100data.shape[0]
print(numrecords)

In [ ]:
# let's convert ncpus NOW
P100data.ncpus = P100data.ncpus/28

In [ ]:
counter = 0
newcounter = 0
P100new = pd.DataFrame(P100data[0:1])  # just to get something in here to initialize the new dataframe
print (P100data.hours)

In [ ]:
print(P100data.ncpus.max())

In [ ]:
while counter < numrecords:
    # the below is to ignore all '0 day' records from original spreadsheet
    #if (P100data.iloc[counter,9] == 0):
     #   P100data.iloc[counter,11] = 0
    thisRecord = P100data[counter:(counter+1)]  
    numhours = P100data.iloc[counter,10]
    # number of days is plain simple to calc using the seconds fields, vs the datatime stuff we do next
    #there are 86400 seconds in a day ; get days using (end-start)/seconds in a day    
    thisRecordNumDays = math.ceil((thisRecord.end - thisRecord.start) / 86400)
    
    if (thisRecordNumDays > 1):        
       
        #numhours = thisRecord.loc['hours']
        print("hours are ")
        print(numhours)
        # NOW decrement original record num hours 
        P100data.iloc[counter, 10] = 24
        # this is now the 'rest' of the hours of the job
        numhours = float(numhours - 24)
        print("hours are ")
        print(numhours)
        # how many more days than 1 day - aka, how many records to add for this instance of a multiday record?
        for i in range(1,(thisRecordNumDays)):
            dateAndTime = str(thisRecord.startdate)
            #print(dateAndTime)
            listDateTime = dateAndTime.split()
            startdatadate = datetime.date.fromisoformat(listDateTime[1])
            startdatatime = datetime.time.fromisoformat(listDateTime[2])
            isodatetime = datetime.datetime.combine(startdatadate, startdatatime)
            nextdate = (isodatetime + timedelta(days=i))             
            #print(nextdate.date)
            dateString = nextdate.strftime("%m/%d/%Y")
            # we now can create our new record for the extra dataframe
            P100new.loc[newcounter] = P100data.iloc[counter]
            # change the date of the added record
            P100new.iloc[newcounter, 7] = dateString
            if (float(numhours) > 24.0):
                P100new.iloc[newcounter,10] = 24
                numhours = float(numhours - 24)
            else:
                #print("hours are ")
                #print(numhours)
                P100new.iloc[newcounter,10] = (numhours)
            
            newcounter = newcounter+1
        print(P100new)        
    counter = counter + 1
    

In [ ]:
# ready to append the two dataframes 
P100alldata = P100data.append(P100new, ignore_index=True)

In [ ]:
P100alldata.shape[0]
print(P100alldata.shape[0])
P100alldata.to_excel('HPC_stats_hoursdiv.xlsx', sheet_name='P100')

In [ ]:
### !!! make ncpus be actually NCPU HOURS 
P100alldata['ncpus'] = P100alldata['ncpus']*P100alldata['hours']
# group the data by the startdate and sum the number of ncpu hours
gimmie = P100alldata.groupby('start_date')['ncpus'].sum()

In [ ]:
print(gimmie)
# these are now ncpu hours per date

In [ ]:
print(gimmie.max())


In [ ]:
gimmie.to_excel('gimmiencpuhours.xlsx', sheet_name='P100')

In [ ]:
# convert these sums from hours of ncpus per day to ncpus per day
gimmie = gimmie / 24
# make this series a dataframe
gimmiedf = gimmie.to_frame().reset_index()



In [ ]:
#p = figure(plot_width=980, plot_height=720, x_axis_type='datetime', y_range=(0,50))


In [ ]:

gimmiedates = [datetime.datetime.strptime(x, '%m/%d/%Y') for x in gimmiedf.start_date.astype(str)]
gimmiedf.start_date = gimmiedates
print(gimmiedf)


In [ ]:
gimmiesort = gimmiedf.sort_values(by=['start_date'])
print(gimmiesort)

In [ ]:
x = gimmiesort['start_date']
y = gimmiesort['ncpus']
line = (x,y)


In [ ]:
# Convert dataframe to column data source for hovertool

src = ColumnDataSource(gimmiesort)
src.data.keys()

#dict_keys(['flights', 'left', 'right', 'index'])

In [ ]:

#hover = HoverTool()

p= figure(plot_width=980,
          plot_height=720,
          title="GPU Node Usage",
          y_range=(0,50),
          x_axis_type="datetime",   
          #tools=[hover, 'tap','box_zoom','wheel_zoom','save','reset'],
         )

p.line(x="start_date", y="ncpus", source=src)

In [ ]:
hover = HoverTool(tooltips = 
                  [('NCPUS', '@ncpus'),
                 ('Date', '@start_date{%Y-%m-%d}')],                    
                 formatters=
                  { 'ncpus': 'printf',
                      'start_date': 'datetime',
                
                  },)

In [ ]:
# Add the hover tool to the graph
p.add_tools(hover)

In [ ]:
show(p)